# 把 order 进行细节的拆分 拆到每一个order的细节 具备





1. order_id
1. order_date
1. order_datetime, 具体的日期，用于分析行为
2. order_user_id，留在后边用
3. order_provice
4. order_city
    这两个部分先按照省份来拆分
5. 


In [5]:
# 2023/02/07 只生成了部分 12月份的数据
# 2023/5 改成生成全部的细节数据 还未完全完成

In [6]:
from imports import *

In [7]:
# read order monthly data 

In [8]:
df = pd.read_csv(os.path.join(GE_DIR,'order_monthly.csv'))

In [9]:
df

,product_id,month,count_raw,year,product_name
0,1,2018-01-01,1057,2018,基础款
1,1,2018-02-01,997,2018,基础款
2,1,2018-03-01,1219,2018,基础款
3,1,2018-04-01,1029,2018,基础款
4,1,2018-05-01,1783,2018,基础款
...,...,...,...,...,...
139,4,2022-08-01,3418,2022,豪华款
140,4,2022-09-01,3460,2022,豪华款
141,4,2022-10-01,4003,2022,豪华款
142,4,2022-11-01,2870,2022,豪华款


In [10]:
week_ratio = [0.8,1,2,2.5,3.8,4,3.5]

In [11]:
# import relativedelta
from dateutil.relativedelta import relativedelta

In [12]:
# Generate random hour and minute according to the ratio of 24 hours

random_24_hour_rato = [1,1,1,1,1,1,2,3,4,8,7,6,3,6,5,4,3,3,2,4,7,8,12,7]
px.scatter(random_24_hour_rato)

In [13]:
# 按照平均，周末更多的方式分配订单


df_list = []
# iterate over df
for idx, row in df.iterrows():
    
    month_str = row['month']
    count_raw = int(row['count_raw'])
    
    first_day = datetime.strptime(month_str, '%Y-%m-%d')
    this_month = first_day.month
    last_day = first_day + relativedelta(months=1, days=-1)
    
    df_this_month = pd.DataFrame()
    dates_of_this_month = pd.date_range(first_day, last_day)
    df_this_month['date'] = dates_of_this_month

    week_days = dates_of_this_month.dayofweek + 1
    df_this_month['week_day'] = week_days
    
    # Generate order count for each week day according to week_ratio
    order_count = [count_raw * ratio / sum(week_ratio) for ratio in week_ratio]
    
    # Generate dict including day of week and order count
    _dict = dict(zip(week_days, order_count))
    
    # number of week days of each week_day
    df_this_month['num_week_day'] = df_this_month['week_day'].map(df_this_month['week_day'].value_counts())
    
    df_this_month['count_total'] = df_this_month['week_day'].map(_dict)
    
    df_this_month['count_orders'] = df_this_month['count_total'] / df_this_month['num_week_day']
    
    # Generate randomness for count_orders follow normal distribution
    df_this_month['count_orders_this_day'] = df_this_month['count_orders'].apply(lambda x: int(np.random.normal(x, x/5)))
    
    df_list.append(df_this_month)


In [14]:
df_this_month

,date,week_day,num_week_day,count_total,count_orders,count_orders_this_day
0,2022-12-01,4,5,169.818182,33.963636,33
1,2022-12-02,5,5,212.272727,42.454545,57
2,2022-12-03,6,5,424.545455,84.909091,92
3,2022-12-04,7,4,530.681818,132.670455,128
4,2022-12-05,1,4,806.636364,201.659091,170
5,2022-12-06,2,4,849.090909,212.272727,220
6,2022-12-07,3,4,742.954545,185.738636,162
7,2022-12-08,4,5,169.818182,33.963636,16
8,2022-12-09,5,5,212.272727,42.454545,38
9,2022-12-10,6,5,424.545455,84.909091,123


In [15]:
df_all = pd.concat(df_list)

In [16]:
df_all

,date,week_day,num_week_day,count_total,count_orders,count_orders_this_day
0,2018-01-01,1,5,48.045455,9.609091,6
1,2018-01-02,2,5,60.056818,12.011364,17
2,2018-01-03,3,5,120.113636,24.022727,28
3,2018-01-04,4,4,150.142045,37.535511,44
4,2018-01-05,5,4,228.215909,57.053977,64
...,...,...,...,...,...,...
26,2022-12-27,2,4,849.090909,212.272727,214
27,2022-12-28,3,4,742.954545,185.738636,248
28,2022-12-29,4,5,169.818182,33.963636,41
29,2022-12-30,5,5,212.272727,42.454545,37


In [17]:
# Generate random hour and minute according to the ratio of 24 hours
def generate_hour_minute():
    hour = np.random.choice(range(24), p=np.array(random_24_hour_rato)/sum(random_24_hour_rato))
    minute = np.random.choice(range(60))
    return hour, minute

# Generate id with random hour and minute

In [18]:
# 
order_list = []

idx = 0
# for idx, row in df_this_month.iterrows():

for idx, row in df_all.iterrows():

    for i in range(row['count_orders_this_day']):
        hour, minute = generate_hour_minute()
        
        order = {
            "id": idx,
            "date": row['date'].strftime('%Y-%m-%d'),
            "time": f"{hour:02d}:{minute:02d}",
            "datetime": row['date'].strftime('%Y-%m-%d') + f" {hour:02d}:{minute:02d}",
            
        }
        
        idx += 1
        
        order_list.append(order)

In [19]:
# change to df
df_order = pd.DataFrame(order_list)

In [20]:
df_order

,id,date,time,datetime
0,0,2018-01-01,06:40,2018-01-01 06:40
1,1,2018-01-01,16:44,2018-01-01 16:44
2,2,2018-01-01,09:08,2018-01-01 09:08
3,3,2018-01-01,22:57,2018-01-01 22:57
4,4,2018-01-01,09:10,2018-01-01 09:10
...,...,...,...,...
1003178,134,2022-12-31,06:04,2022-12-31 06:04
1003179,135,2022-12-31,09:04,2022-12-31 09:04
1003180,136,2022-12-31,21:10,2022-12-31 21:10
1003181,137,2022-12-31,21:39,2022-12-31 21:39


# 获取省份





In [21]:
# Generate list of all provices in China
provices = ['北京市','天津市','河北省','山西省','内蒙古自治区','辽宁省','吉林省','黑龙江省','上海市','江苏省','浙江省','安徽省','福建省','江西省','山东省','河南省','湖北省','湖南省','广东省','广西壮族自治区','海南省','重庆市','四川省','贵州省','云南省','西藏自治区','陕西省','甘肃省','青海省','宁夏回族自治区','新疆维吾尔自治区','台湾省','香港特别行政区','澳门特别行政区']

In [22]:
# generate all GDP data of all provices

# read order_propotion data

df_order_propotion = pd.read_excel(os.path.join(BASE_DIR,'order_propotion.xlsx'))

In [23]:
df_order_propotion

,provice,propotion
0,北京,0.10
1,上海,0.20
2,广州,0.11
3,深圳,0.10
4,江苏,0.30
5,浙江,0.25
6,山东,0.10


In [24]:
# generate n random choise from list df_order_propotion['provice']
import random

# generate n random choise from list df_order_propotion['provice'] with probabiliy of df_order_propotion['propotion']
import random

# generate n random choices from list df_order_propotion['province'] with probability of df_order_propotion['proportion']
n = 100
provinces = df_order_propotion['provice']
random_choices = random.choices(provinces, weights=df_order_propotion['propotion'], k=n)

# print the random choices
print(random_choices)


['江苏', '浙江', '江苏', '上海', '江苏', '江苏', '浙江', '深圳', '江苏', '浙江', '深圳', '北京', '山东', '北京', '上海', '浙江', '江苏', '深圳', '浙江', '山东', '浙江', '上海', '广州', '上海', '上海', '上海', '浙江', '深圳', '上海', '浙江', '浙江', '深圳', '浙江', '江苏', '浙江', '江苏', '浙江', '深圳', '江苏', '深圳', '江苏', '上海', '江苏', '广州', '浙江', '江苏', '山东', '上海', '浙江', '北京', '浙江', '深圳', '山东', '浙江', '山东', '深圳', '江苏', '浙江', '上海', '上海', '上海', '山东', '上海', '深圳', '山东', '深圳', '山东', '江苏', '山东', '江苏', '上海', '上海', '浙江', '上海', '山东', '北京', '江苏', '深圳', '江苏', '浙江', '北京', '上海', '上海', '上海', '江苏', '山东', '山东', '北京', '上海', '上海', '北京', '上海', '广州', '江苏', '江苏', '上海', '江苏', '江苏', '深圳', '广州']


In [25]:
# generate random provice base on the propotion

def generate_provice(x):
    # make sum of propotion equal to 1
    df_order_propotion['propotion'] = df_order_propotion['propotion'] / sum(df_order_propotion['propotion'])
    
    provice = np.random.choice(df_order_propotion['provice'], p=df_order_propotion['propotion'].values)
    return provice

def generate_provice_m(n):
    # n = 100
    provinces = df_order_propotion['provice']
    random_choices = random.choices(provinces, weights=df_order_propotion['propotion'], k=n)

    # print the random choices
    # print(random_choices)
    return random_choices

In [26]:
# df_order['provice'] = df_order['id'].apply(generate_provice)

province_generated = generate_provice_m(df_order.shape[0])


df_order['provice'] = province_generated

In [25]:
df_order

,id,date,time,datetime,provice
0,0,2018-01-01,22:14,2018-01-01 22:14,江苏
1,1,2018-01-01,11:10,2018-01-01 11:10,浙江
2,2,2018-01-01,01:06,2018-01-01 01:06,江苏
3,3,2018-01-01,10:30,2018-01-01 10:30,浙江
4,4,2018-01-01,20:48,2018-01-01 20:48,上海
...,...,...,...,...,...
998932,106,2022-12-31,22:52,2022-12-31 22:52,江苏
998933,107,2022-12-31,02:53,2022-12-31 02:53,浙江
998934,108,2022-12-31,06:21,2022-12-31 06:21,浙江
998935,109,2022-12-31,16:01,2022-12-31 16:01,浙江


In [27]:
#
# df_order.groupby('provice').count()

In [28]:
df_order

,id,date,time,datetime,provice
0,0,2018-01-01,22:14,2018-01-01 22:14,江苏
1,1,2018-01-01,11:10,2018-01-01 11:10,浙江
2,2,2018-01-01,01:06,2018-01-01 01:06,江苏
3,3,2018-01-01,10:30,2018-01-01 10:30,浙江
4,4,2018-01-01,20:48,2018-01-01 20:48,上海
...,...,...,...,...,...
998932,106,2022-12-31,22:52,2022-12-31 22:52,江苏
998933,107,2022-12-31,02:53,2022-12-31 02:53,浙江
998934,108,2022-12-31,06:21,2022-12-31 06:21,浙江
998935,109,2022-12-31,16:01,2022-12-31 16:01,浙江


In [4]:
# read city data

df_city = pd.read_excel('../map_generation_src/省市县区.xlsx')

NameError: name 'pd' is not defined

In [30]:
# remove "省" and "市" from provice
df_city['provice'] = df_city['provice'].apply(lambda x: x.replace('省','').replace('市',''))

In [31]:
df_city['provice'].unique()

array(['北京', '天津', '河北', '山西', '内蒙古', '辽宁', '吉林', '黑龙江', '上海', '江苏', '浙江',
       '安徽', '福建', '江西', '山东', '河南', '湖北', '湖南', '广东', '广西', '海南', '重庆',
       '四川', '贵州', '云南', '西藏', '陕西', '甘肃', '青海', '宁夏', '新疆'], dtype=object)

In [32]:
df_city

,provice,city,县区
0,北京,北京市,东城区
1,北京,北京市,西城区
2,北京,北京市,崇文区
3,北京,北京市,宣武区
4,北京,北京市,朝阳区
...,...,...,...
2861,新疆,阿勒泰地区,吉木乃县
2862,新疆,省直辖行政单位,石河子市
2863,新疆,省直辖行政单位,阿拉尔市
2864,新疆,省直辖行政单位,图木舒克市


In [3]:
df_city

NameError: name 'df_city' is not defined

In [2]:
# Generate random city of each provice
# too slow
def generate_city(x,df_city = df_city):
    try:
        df_this_provice = df_city[df_city['provice'] == x]
        city = np.random.choice(df_this_provice['city'])
    except:
        city = x
    return city


def generate_city(n,p,df_city = df_city):
    # n = 100
    df_this_provice = df_city[df_city['provice'] == p]
    city = np.random.choice(df_this_provice['city'], k = n)
    

    # print the random choices
    # print(random_choices)
    return random_choices

NameError: name 'df_city' is not defined

In [ ]:
#

for p in df_order['provice'].unique():
    
    df_p = df_order[df_order['provice'] == p]
    
    # generate city for this p
    city_generated = generate_city(df_p.shape[0])

In [45]:
df_order['city'] = df_order['provice'].apply(generate_city)

In [33]:
df_order

,id,date,time,datetime,provice
0,0,2018-01-01,22:14,2018-01-01 22:14,江苏
1,1,2018-01-01,11:10,2018-01-01 11:10,浙江
2,2,2018-01-01,01:06,2018-01-01 01:06,江苏
3,3,2018-01-01,10:30,2018-01-01 10:30,浙江
4,4,2018-01-01,20:48,2018-01-01 20:48,上海
...,...,...,...,...,...
998932,106,2022-12-31,22:52,2022-12-31 22:52,江苏
998933,107,2022-12-31,02:53,2022-12-31 02:53,浙江
998934,108,2022-12-31,06:21,2022-12-31 06:21,浙江
998935,109,2022-12-31,16:01,2022-12-31 16:01,浙江


In [103]:
# save as csv
df_order.to_csv(os.path.join(GE_DIR,'order_detail.csv'), index=False)